In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import pkg_resources
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import gymnasium as gym
import stable_baselines3 as sb3
import stable_baselines3.common.logger as logger
import stable_baselines3.common.callbacks as callbacks
import stable_baselines3.common.env_checker as env_checker
from dm_control import mjcf
from dm_control.rl.control import PhysicsError
import imageio
import scipy.spatial
import torch
import torch.nn as nn
import torch.optim as optim
import torch_geometric as pyg
import torch.nn.functional as F
import torch_geometric.nn as gnn
import torch_geometric.loader as pyg_loader
import pytorch_lightning as pl
import torchmetrics
from torch.utils.data import Dataset
from pathlib import Path
from typing import Tuple, Callable, Optional, List, Union
from tqdm import trange
from dm_control.rl.control import PhysicsError
from PIL import Image

from flygym.arena.mujoco_arena import FlatTerrain
from flygym.envs.nmf_mujoco import NeuroMechFlyMuJoCo, MuJoCoParameters
from flygym.state import stretched_pose
import flygym.util.vision as vision
import flygym.util.config as config
from flygym.arena import BaseArena
from flygym.arena.mujoco_arena import OdorArena, FlatTerrain, GappedTerrain, BlocksTerrain
from flygym.util.data import color_cycle_rgb
from flygym.util.turning_controller import TurningController

from vision_model import VisualFeaturePreprocessor
from navigation_arena import ObstacleOdorArena

pygame 2.5.1 (SDL 2.28.2, Python 3.11.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):
/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/lightning_fabric/__init__.py:36: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('lightning_fabric')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  __import__("pkg_resources").declare_namespace(__name__)
/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Load retina graph

In [4]:
with open("data/ommatidia_graph.pkl", "rb") as f:
    ommatidia_graph_nx = pickle.load(f)
ommatidia_graph_pg = pyg.utils.from_networkx(ommatidia_graph_nx)

Load visual feature extraction model

In [5]:
vision_model = VisualFeaturePreprocessor.load_from_checkpoint(
    "data/models/visual_preprocessor.pt"
)

Define MDP task

In [6]:
class NMFNavigation(TurningController):
    def __init__(
        self,
        arena,
        vision_model,
        ommatidia_graph,
        device=torch.device("cpu"),
        decision_dt=0.1,
        n_stabilisation_dur=0.3,
        distance_threshold=15,
        max_time=5,
        test_mode=False,
        debug_mode=False,
        **kwargs,
    ) -> None:
        self.debug_mode = debug_mode
        sim_params = MuJoCoParameters(
            render_playspeed=0.1,
            render_camera="birdeye_cam",
            enable_vision=True,
            render_raw_vision=test_mode,
            enable_olfaction=True,
            render_mode="saved" if test_mode else "headless",
            vision_refresh_rate=int(1 / decision_dt),
        )
        super().__init__(
            sim_params=sim_params,
            arena=arena,
            stabilisation_dur=n_stabilisation_dur,
            detect_flip=True,
            **kwargs,
        )

        self.device = device
        self.ommatidia_graphs = [ommatidia_graph.clone(), ommatidia_graph.clone()]
        self.vision_model = vision_model.to(self.device)
        self.max_time = max_time
        self.arena = arena
        self.num_substeps = int(decision_dt / self.timestep)
        self.distance_threshold = distance_threshold

        # Override spaces
        # action space: 2D vector of amplitude and phase for oscillators on each side
        # observation space:
        #  - 2D vector of x-y position of object relative to the fly, norm. to [0, 1]
        #  - scalar probability that there is an object in view, [0, 1]
        #  - 2D vector of mean odor intensity on each side, norm. to [0, 1]
        #  - 2D vector of current oscillator amp. on each side, norm. to [0, 1]
        self.action_space = gym.spaces.Box(low=-1, high=1, shape=(2,))
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(7,))

        self._last_fly_tgt_dist = np.linalg.norm(
            np.zeros(2) - self.arena.odor_source[0, :2]
        )
        self._last_action = np.zeros((2,))

    def update_retina_graphs(self, intensities):
        intensities = torch.tensor(intensities, dtype=torch.float32).sum(axis=-1) / 255
        intensities = intensities.to(self.device)
        for i in range(2):
            self.ommatidia_graphs[i].x = intensities[i, :]

    def step(self, amplitude):
        try:
            obstacle_contact_counter = 0
            for i in range(self.num_substeps):
                raw_obs, _, raw_term, raw_trunc, info = super().step(amplitude)
                collision_forces = [
                    np.abs(self.physics.named.data.cfrc_ext[f"obstacle_{j}"]).sum()
                    for j in range(len(self.arena.obstacle_positions))
                ]
                if np.sum(collision_forces) > 1:
                    obstacle_contact_counter += 1
                super().render()
            collision = obstacle_contact_counter > 0.5 * self.num_substeps
        except PhysicsError:
            print("Physics error, resetting environment")
            return np.zeros((7,), dtype="float32"), 0, False, True, {}

        # Check if visual inputs are rendered recently
        assert abs(self.curr_time - self._last_vision_update_time) < 0.5 * self.timestep

        # Parse observations
        self.update_retina_graphs(self.curr_visual_input)
        pos_pred, obj_prob = self.vision_model(*self.ommatidia_graphs)
        pos_pred = pos_pred.detach().numpy().squeeze() / self.distance_threshold
        pos_pred = np.clip(pos_pred, 0, 1)
        obj_prob = obj_prob.detach().numpy()
        odor_intensity = raw_obs["odor_intensity"][0, :].reshape(2, 2).mean(axis=0)
        odor_intensity /= self.arena.peak_odor_intensity[0, 0]
        last_action = self._last_action / 2 + 0.5
        obs = np.concatenate(
            [pos_pred, obj_prob, odor_intensity, last_action], dtype=np.float32
        )

        # Calculate reward
        # calculate distance reward
        fly_pos = super().get_observation()["fly"][0, :2]
        tgt_pos = self.arena.odor_source[0, :2]
        distance = np.linalg.norm(fly_pos - tgt_pos)
        distance_reward = self._last_fly_tgt_dist - distance
        self._last_fly_tgt_dist = distance

        # check if fly is too close to any obstacle
        has_collision = False
        for obst_pos in self.arena.obstacle_positions:
            if np.linalg.norm(fly_pos - obst_pos) < self.arena.obstacle_radius + 1:
                has_collision = True
                break

        # calculate final reward
        if distance < 2:
            reward = 30
            terminated = True
            info["state_desc"] = "success"
        elif collision:
            reward = -5
            terminated = True
            info["state_desc"] = "collision"
        elif info["flip"]:
            reward = -5
            terminated = True
            info["state_desc"] = "flipped"
        else:
            reward = distance_reward
            terminated = False
            info["state_desc"] = "seeking"
        
        # apply penalty for rapid turning
        action_diff = np.abs(amplitude - self._last_action).sum() / 2

        info["distance_reward"] = distance_reward
        info["has_collision"] = has_collision
        info["distance"] = distance
        truncated = (
            self.curr_time > self.max_time and not terminated
        )  # start a new episode

        if self.debug_mode:
            print(f"fly_pos: {fly_pos}, reward={reward}, state={info['state_desc']}")
            if terminated:
                print("terminated")
            if truncated:
                print("truncated")
        
        self._last_action = amplitude
        return obs, reward, terminated, truncated, info

    def reset(self, seed=0):
        super().reset()
        obs = np.array([0, 0, 0, 0, 0, 0.5, 0.5], dtype="float32")
        self._last_fly_tgt_dist = np.linalg.norm(
            np.zeros(2) - self.arena.odor_source[0, :2]
        )
        self._last_action = np.zeros((2,))
        if self.debug_mode:
            print("resetting environment")
        return obs, {"state_desc": "reset"}

In [7]:
terrain_arena = FlatTerrain(ground_alpha=1)
arena = ObstacleOdorArena(
    terrain=terrain_arena,
    obstacle_positions=np.array([(7.5, -2.5), (12.5, 2.5)]),
    odor_source=np.array([[20, 0, 2]]),
    marker_size=0.5,
    obstacle_colors=(0.14, 0.14, 0.2, 1),
)
sim = NMFNavigation(
    spawn_pos=np.array([5, 0, 0]),
    arena=arena,
    vision_model=vision_model,
    ommatidia_graph=ommatidia_graph_pg,
    test_mode=True,
    debug_mode=True,
)
env_checker.check_env(sim)

/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


resetting environment
resetting environment
resetting environment
fly_pos: [ 3.9329195  -0.10393089], reward=3.932663140450657, state=seeking
resetting environment
fly_pos: [4.38542067 0.00805808], reward=4.385419093901756, state=seeking
fly_pos: [5.00995423 0.1175348 ], reward=0.6241896037907111, state=seeking
fly_pos: [ 5.30737244 -0.22641765], reward=0.29646216193157926, state=seeking
fly_pos: [5.5954696  0.01594871], reward=-10, state=collision
terminated
resetting environment
fly_pos: [ 4.28742342 -0.14806542], reward=4.2868941967232175, state=seeking
fly_pos: [ 5.02696505 -0.0588084 ], reward=0.7399842772924003, state=seeking
fly_pos: [5.37436605 0.1219977 ], reward=0.34710840095400286, state=seeking
fly_pos: [ 5.54695655 -0.06689521], reward=-10, state=collision
terminated
resetting environment
fly_pos: [ 4.21534329 -0.26079222], reward=4.213707230717596, state=seeking
fly_pos: [ 4.2685994  -0.44876242], reward=0.05003566780369795, state=seeking


In [8]:
sim.reset()
for i in trange(10):
    obs, reward, terminated, truncated, info = sim.step(np.ones((2,)) * 0.8)
    if terminated:
        break

resetting environment


 10%|█         | 1/10 [00:02<00:24,  2.70s/it]

fly_pos: [ 4.22271406 -0.03561077], reward=4.222683541946729, state=seeking


 20%|██        | 2/10 [00:05<00:21,  2.63s/it]

fly_pos: [4.80596661 0.32148532], reward=0.5807242334746832, state=seeking


 30%|███       | 3/10 [00:07<00:17,  2.54s/it]

fly_pos: [4.79460167 0.90645849], reward=-0.029128747016116563, state=seeking


 30%|███       | 3/10 [00:10<00:24,  3.43s/it]

fly_pos: [5.92755868 0.92293877], reward=-10, state=collision
terminated


In [9]:
sim.save_video("test.mp4", stabilization_time=0)

In [10]:
terrain_arena = FlatTerrain(ground_alpha=1)
arena = ObstacleOdorArena(
    terrain=terrain_arena,
    obstacle_positions=np.array([(7.5, 0), (12.5, 5), (17.5, -5)]),
    odor_source=np.array([[25, 0, 2]]),
    marker_size=0.5,
    obstacle_colors=(0.14, 0.14, 0.2, 1),
)
sim = NMFNavigation(
    arena=arena,
    vision_model=vision_model,
    ommatidia_graph=ommatidia_graph_pg,
    max_time=5,
    test_mode=False,
)
env_checker.check_env(sim)

np.random.seed(0)
sb3.common.utils.set_random_seed(0, using_cuda=True)

start_from = None
train = True

log_dir = "logs/trial_3"
checkpoint_callback = callbacks.CheckpointCallback(
    save_freq=1000,
    save_path=log_dir,
    name_prefix="trial_3",
    save_replay_buffer=True,
    save_vecnormalize=True,
)
my_logger = logger.configure(log_dir, ["tensorboard", "stdout", "csv"])
model = sb3.SAC(
    "MlpPolicy",
    env=sim,
    policy_kwargs={"net_arch": [32, 32]},
    verbose=2,
    learning_rate=0.01,
)
if start_from is not None:
    model = sb3.SAC.load(start_from)
model.set_logger(my_logger)

if train:
    model.learn(total_timesteps=50_000, progress_bar=True, callback=checkpoint_callback)
    model.save("models/trial_3")

Logging to logs/trial_2


Output()

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 13       |
|    ep_rew_mean     | -5.5     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 0        |
|    time_elapsed    | 69       |
|    total_timesteps | 52       |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 12       |
|    ep_rew_mean     | -6.08    |
| time/              |          |
|    episodes        | 8        |
|    fps             | 0        |
|    time_elapsed    | 127      |
|    total_timesteps | 96       |
---------------------------------


...

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 35.4     |
|    ep_rew_mean     | -7.8     |
| time/              |          |
|    episodes        | 172      |
|    fps             | 0        |
|    time_elapsed    | 6664     |
|    total_timesteps | 5231     |
| train/             |          |
|    actor_loss      | 6.23     |
|    critic_loss     | 1.72     |
|    ent_coef        | 0.0238   |
|    ent_coef_loss   | -0.0745  |
|    learning_rate   | 0.01     |
|    n_updates       | 5130     |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 35.6     |
|    ep_rew_mean     | -8.35    |
| time/              |          |
|    episodes        | 176      |
|    fps             | 0        |
|    time_elapsed    | 6796     |
|    total_timesteps | 5323     |
| train/             |          |
|    actor_loss      | 6.41     |
|    critic_loss     | 1.8      |
|    ent_coef        | 0.0147   |
|    ent_coef_loss   | -0.484   |
|    learning_rate   | 0.01     |
|    n_updates       | 5222     |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 34.5     |
|    ep_rew_mean     | -8.28    |
| time/              |          |
|    episodes        | 180      |
|    fps             | 0        |
|    time_elapsed    | 6861     |
|    total_timesteps | 5373     |
| train/             |          |
|    actor_loss      | 6.01     |
|    critic_loss     | 2.36     |
|    ent_coef        | 0.0129   |
|    ent_coef_loss   | 1.03     |
|    learning_rate   | 0.01     |
|    n_updates       | 5272     |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 34.4     |
|    ep_rew_mean     | -8.18    |
| time/              |          |
|    episodes        | 184      |
|    fps             | 0        |
|    time_elapsed    | 6982     |
|    total_timesteps | 5464     |
| train/             |          |
|    actor_loss      | 6.71     |
|    critic_loss     | 1.18     |
|    ent_coef        | 0.0373   |
|    ent_coef_loss   | 1.03     |
|    learning_rate   | 0.01     |
|    n_updates       | 5363     |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 34.1     |
|    ep_rew_mean     | -8.06    |
| time/              |          |
|    episodes        | 188      |
|    fps             | 0        |
|    time_elapsed    | 7037     |
|    total_timesteps | 5504     |
| train/             |          |
|    actor_loss      | 6.78     |
|    critic_loss     | 1.15     |
|    ent_coef        | 0.0462   |
|    ent_coef_loss   | -0.945   |
|    learning_rate   | 0.01     |
|    n_updates       | 5403     |
---------------------------------


In [ ]:
np.abs(sim.physics.named.data.cfrc_ext["obstacle_0"]).sum()

23.330371908742478